#DATASCI W261: Machine Learning at Scale

Nick Hamlin and Tigi Thomas  
nickhamlin@gmail.com, tgthomas@berkeley.edu   
Time of Submission: 9:23 PM EST, Wednesday, Feb 10, 2016  
W261-3, Spring 2016  
Week 4 Homework

###Submission Notes:
- For each problem, we've included a summary of the question as posed in the instructions.  In many cases, we have not included the full text to keep the final submission as uncluttered as possible.  For reference, we've included a link to the original instructions in the "Useful Reference" below.
- Problem statements are listed in *italics*, while our responses are shown in plain text. 
- We've included the full output of the hadoop jobs in our responses so that counter results are shown.  However, these don't always render nicely into PDF form.  In these situations, please reference [the complete rendered notebook on Github](https://github.com/nickhamlin/mids_261_homework/blob/master/HW3/MIDS-W261-2015-HWK-Week03-Hamlin-Thomas.ipynb)

###Useful References:
- **[Original Assignment Instructions](https://www.dropbox.com/sh/m0nxsf4vs5cyrp2/AACYOZQ3hRyGtHoPt33ny_Pza/HW4-Questions.txt?dl=0)**
- [Most frequent word example in mrjob](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/nd2wow1t3y77jqk/MrjobMostUsedWord.ipynb)


##HW4.0.  
*What is MrJob? How is it different to Hadoop MapReduce?*  

*What are the mappint_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?*  



##HW4.1

*What is serialization in the context of MrJob or Hadoop?*  

*When it used in these frameworks?*  

*What is the default serialization mode for input and outputs for MrJob?*  

## HW 4.2

Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

 Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

C,"10001",10001   #Visitor id 10001  
V,1000,1          #Visit by Visitor 10001 to page id 1000  
V,1001,1          #Visit by Visitor 10001 to page id 1001  
V,1002,1          #Visit by Visitor 10001 to page id 1002  
C,"10002",10002   #Visitor id 10001  
V  
Note: #denotes comments
to the format:

V,1000,1,C, 10001  
V,1001,1,C, 10001  
V,1002,1,C, 10001  

Write the python code to accomplish this.

In [28]:
%%writefile convert_msdata.py

from csv import reader
with open('anonymous-msweb.data','rb') as f:
    data=f.readlines()
    
for i in reader(data):
    if i[0]=='C':
        visitor_id=i[1]
        continue
    if i[0]=='V':
        print i[0]+','+i[1]+','+i[2]+',C,'+visitor_id

Overwriting convert_msdata.py


In [127]:
%%writefile create_urls.py

from csv import reader
with open('anonymous-msweb.data','rb') as f:
    data=f.readlines()
    
for i in reader(data):
    if i[0]=='A':
        print i[1]+','+i[3]+','+i[4]

Writing create_urls.py


In [128]:
#Make files executable, convert data, view some results
#!chmod +x convert_msdata.py create_urls.py
!python convert_msdata.py > clean_msdata.txt
!cat clean_msdata.txt | head -10
!python create_urls.py > ms_urls.txt
!cat ms_urls.txt | head -10

V,1000,1,C,10001
V,1001,1,C,10001
V,1002,1,C,10001
V,1001,1,C,10002
V,1003,1,C,10002
V,1001,1,C,10003
V,1003,1,C,10003
V,1004,1,C,10003
V,1005,1,C,10004
V,1006,1,C,10005
cat: stdout: Broken pipe
1287,International AutoRoute,/autoroute
1288,library,/library
1289,Master Chef Product Information,/masterchef
1297,Central America,/centroam
1215,For Developers Only Info,/developer
1279,Multimedia Golf,/msgolf
1239,Microsoft Consulting,/msconsult
1282,home,/home
1251,Reference Support,/referencesupport
1121,Microsoft Magazine,/magazine


## HW 4.3
*Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).*

In [18]:
%load_ext autoreload
%autoreload 2

In [125]:
%%writefile top_pages.py
"""
This program will take a CSV data file and output tab-seperated lines of

    Vroot -> number of visits

To run:

    python top_pages.py anonymous-msweb.data

To store output:

    python top_pages.py anonymous-msweb.data > top_pages.out
"""
from mrjob.job import MRJob
import csv

def csv_readline(line):
    """Given a string CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

class TopPages(MRJob):
    
# Normally, we'd use the shuffle to do the sort.  However, the bug
# in comparitors when running local MRJobs makes this an untenable solution
# so we'll settle for doing the sort in the second-stage reducer instead

#     def jobconf(self):
#         orig_jobconf = super(TopPages, self).jobconf()        
#         custom_jobconf = {  #key value pairs
#             'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
#             'mapred.text.key.comparator.options': '-k1,1nr',
#             'mapred.reduce.tasks': '1',
#         }
#         combined_jobconf = orig_jobconf
#         combined_jobconf.update(custom_jobconf)
#         self.jobconf = combined_jobconf
#         return combined_jobconf

    def mapper_extract_views(self, line_no, line):
        """Extracts the Vroot that was visited"""
        cell = csv_readline(line)
        if cell[0] == 'V':
            yield cell[1],1

    def reducer_sum_views(self, vroot, visit_counts):
        """Sumarizes the visit counts by adding them together,yield the results"""
        
        total = sum(i for i in visit_counts)
        yield None,(total, vroot)
        
        
    # discard the key; it is just None
    def reducer_find_top_views(self,_, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word

        output=sorted(word_count_pairs)[-5:]
        output.reverse()
        for i in output:
            yield (i[1],i[0])
        
        
    def steps(self):  #pipeline of Map-Reduce jobs
        return [
            self.mr(mapper=self.mapper_extract_views,       # STEP 1: view count step
                    reducer=self.reducer_sum_views) ,
            self.mr(reducer=self.reducer_find_top_views) # Step 2: sort and return top 5 results
        ]
        
if __name__ == '__main__':
    TopPages.run()

Overwriting top_pages.py


In [126]:
from top_pages import TopPages
import csv

mr_job = TopPages(args=['clean_msdata.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

('1008', 10836)
('1034', 9383)
('1004', 8463)
('1018', 5330)
('1017', 5108)


In [112]:
test=[('1','a'),('3','f'),('2','s'),('87','ab'),('11','a')]
output=[(int(i[0]),i[1]) for i in test]
output.sort()
output.reverse()
print output

[(87, 'ab'), (11, 'a'), (3, 'f'), (2, 's'), (1, 'a')]


In [33]:
!chmod +x top_pages.py

In [ ]:
!python mr_wc.py mr_wc.py

In [166]:
from collections import Counter
from operator import itemgetter
test=Counter()
visits=[1,1,2,2,2,5,5,5,4]
for i in visits:
    test[i]+=1
    
output= max(test.iteritems(), key=itemgetter(1))[0]

print test
print output
test[output]

Counter({2: 3, 5: 3, 1: 2, 4: 1})
2


3

## HW 4.4

*Find the most frequent visitor of each page using MrJob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.*

In [106]:

print url_dict[1002]


/athome


In [105]:
with open('ms_urls.txt','rb') as f:
    urls=csv.reader(f.readlines())
url_dict={}
for i in urls:
    url_dict[int(i[0])]=i[2]
            
            
            

In [121]:
%%writefile freq_visitor.py

import csv
from collections import Counter
from operator import itemgetter

from mrjob.job import MRJob
from mrjob.step import MRStep


def csv_readline(line):
    """Given a string CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

class FreqVisitor(MRJob):
    
    def jobconf(self):
        orig_jobconf = super(FreqVisitor, self).jobconf()        
        custom_jobconf = {'upload_files': 'ms_urls.txt'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf

    def mapper_extract_views(self, line_no, line):
        """Extracts the visitor id and the vroot that was visited"""
        cell = csv_readline(line)
        if cell[0] == 'V':
            yield cell[4],cell[1]
    
    def reducer_load_urls(self):
        with open('ms_urls.txt','rb') as f:
            urls=csv.reader(f.readlines())
        self.url_dict={}
        for i in urls:
            self.url_dict[int(i[0])]=i[2]

    def reducer_sum_views_by_visitor(self, visitor, vroots):
        """Summarizes page counts for each visitor, 
        yields one record per visitor with the page containing 
        the most views by that visitor"""
        pages=Counter()
        for i in vroots:
            pages[i]+=1
        output= max(pages.iteritems(), key=itemgetter(1))[0]
        yield ('Visitor ID:'+str(visitor)),(output,pages[output],self.url_dict[int(output)])
   
    def steps(self):
        return [MRStep(mapper=self.mapper_extract_views,
                        reducer_init=self.reducer_load_urls,
                        reducer=self.reducer_sum_views_by_visitor)]
        
if __name__ == '__main__':
    FreqVisitor.run()

Overwriting freq_visitor.py


In [122]:
from freq_visitor import FreqVisitor
import csv

mr_job = FreqVisitor(args=['clean_msdata.txt','--file','ms_urls.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

('Visitor ID:10001', ['1002', 1, '/athome'])
('Visitor ID:10002', ['1003', 1, '/kb'])
('Visitor ID:10003', ['1004', 1, '/search'])
('Visitor ID:10004', ['1005', 1, '/norge'])
('Visitor ID:10005', ['1006', 1, '/misc'])
('Visitor ID:10006', ['1004', 1, '/search'])
('Visitor ID:10007', ['1007', 1, '/ie_intl'])
('Visitor ID:10008', ['1004', 1, '/search'])
('Visitor ID:10009', ['1009', 1, '/windows'])
('Visitor ID:10010', ['1014', 1, '/officefreestuff'])
('Visitor ID:10011', ['1015', 1, '/msexcel'])
('Visitor ID:10012', ['1021', 1, '/visualc'])
('Visitor ID:10013', ['1022', 1, '/truetype'])
('Visitor ID:10014', ['1023', 1, '/spain'])
('Visitor ID:10015', ['1024', 1, '/iis'])
('Visitor ID:10016', ['1025', 1, '/gallery'])
('Visitor ID:10017', ['1026', 1, '/sitebuilder'])
('Visitor ID:10018', ['1004', 1, '/search'])
('Visitor ID:10019', ['1004', 1, '/search'])
('Visitor ID:10020', ['1034', 1, '/ie'])
('Visitor ID:10021', ['1040', 1, '/office'])
('Visitor ID:10022', ['1004', 1, '/search'])
('Vi

## HW 4.5


Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of our recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes.

and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt

In [509]:
from __future__ import division
num=2.0
centroid=np.array([1.1,2.2,3.3,3.2,1.1,3.1,4.0])
centroid/=num

    
print centroid
with open('Centroids.txt','w') as f:
    f.writelines(','.join(map(str,centroid))+'\n')
test=[s for s in open('Centroids.txt').readlines()]

print test
open('Centroids.txt','w').close() #This wipes the file!
test2=[s for s in open('Centroids.txt').readlines()]
print test2

[ 0.55  1.1   1.65  1.6   0.55  1.55  2.  ]
['0.55,1.1,1.65,1.6,0.55,1.55,2.0\n']
[]


In [510]:
%%writefile mrkmeans.py
from __future__ import division
from math import sqrt
from operator import itemgetter
from collections import Counter

import numpy as np

from mrjob.job import MRJob
from mrjob.step import MRStep

# def find_distance(point1,point2):
#     """
#     Given two points in multidimensional euclidean space, return the distance between them
#     """
#     results=[]
#     for i in zip(point1,point2):
#         results.append((i[1]-i[0])**2)
#     dist=sqrt(sum(map(float,results)))
#     return dist


# def MinDist2(point_to_test,centroids):
#     """
#     - point_to_test is a tuple
#     - centroids contains a list of tuples to compare the point to
#     returns the index of centroid from list that is closest to the test point
#     """
#     results=[]
#     for v in centroids:
#         #By using a separate distance function, it's easy to swap out our distance metric
#         #if we discover that Euclidean distance doesn't make sense
#         results.append(find_distance(point_to_test,v))
#     min_index, min_value = min(enumerate(results), key=itemgetter(1))
#     return min_index


#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = np.array(datapoint)
    centroid_points = np.array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = np.argmin(list(diffsq.sum(axis = 1)))
    return minidx

class MRKmeans(MRJob):
    centroid_points=[]
    k=4
    
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper
                ,combiner=self.combiner
                ,reducer=self.reducer
                ,reducer_final=self.reducer_final
            )
        ]
    
    def mapper_init(self):
        """
        Load locations of existing centroids into memory as a list with len=k of lists with len=1000
        """
        
        self.centroid_points=[map(float,s.split('\n')[0].split(',')) for s in open('Centroids.txt').readlines()]
        open('Centroids.txt','w').close() #This wipes the file!
        
    def mapper(self,_,line):
        """
        For each point sent through the stream:
        - Normalize each point by the total number of words in the document
        - Calculate the closest centroid
        - Emit records where... 
            -Key=(<current cluster asst>,<correct cluster asst>)
            -Value=(1,<normalized vector for that point>)
        """
        
        line=line.split(',')
        line_id,cluster,total_words=int(line[0]),int(line[1]),float(line[2])
        D=(map(float,line[3:]))
        D=[i/total_words for i in D]
        idx=int(MinDist(D,self.centroid_points))
        class_counts=np.zeros(self.k)
        class_counts[cluster]+=1
        yield idx,(list(class_counts),1,D)
        
    def combiner(self,idx,inputdata):
        """
        For each row sent by the mapper, calculate partial sum for new centroid:
        - Initialize a blank 1000 element list
        - Add all intermediate values together for that list
        - Emit records where...
            -Key=Index of centroid that should be updated with the associated vector
            -Value=(<number of points represented in the vector>,<vector of partial sums>)
        """
        
        temp_row=np.zeros(1000)
        num=0
        class_counts=np.zeros(self.k)
        for v in inputdata:
            class_counts+=v[0] #records will come in with a real cluster id, we'll pass the lists through here
            num+=v[1]
            temp_row+=v[2]
        yield idx,(list(class_counts),num,list(temp_row))
    
    def reducer(self,idx,inputdata):
        """
        For each incoming row:
        - Calculate final sum of vector elements using the same approach as in the combiner
        - Divide by the number of points in the cluster to calculate the updated location of each new centroid
        - Store updated centroids to disk
        - Emit location of new centroids
        """
        centroid=np.zeros(1000)
        class_counts=np.zeros(self.k)
        num=0

        for v in inputdata:
            class_counts+=v[0]
            num+=v[1]
            centroid+=v[2]
        
        centroid/=num
        
        with open('Centroids.txt','a') as f:
            f.writelines(','.join(map(str,centroid))+'\n')
        yield idx,(list(class_counts),list(centroid))

if __name__=='__main__':
    MRKmeans.run()

Overwriting mrkmeans.py


In [514]:
from __future__ import division
from csv import reader
from time import sleep
import random as rand #avoid namespace collision
from itertools import chain

from numpy import random

from mrkmeans import MRKmeans #, find_distance

source='topUsers_Apr-Jul_2014_1000-words.txt'
#source='users_test.txt'


mr_job=MRKmeans(args=[source,'--file', 'Centroids.txt'])

#Generate initial centroids
centroid_points=[]
centroid_dimensions=1000
k=4

#Part A - choose 4 random users to initialize cluster
users=(rand.sample(list(open(source)),k))
for line in reader(users):
    line_id,cluster,total_words=int(line[0]),int(line[1]),float(line[2])
    D=(map(float,line[3:]))
    D=[i/total_words for i in D]
    centroid_points.append(D)
    
#for i in range(k): #THIS IS OLD
#    centroid_points.append([random.uniform(-.01,.01) for i in range(centroid_dimensions)])
with open('Centroids.txt','w+') as f:
    f.writelines(','.join(str(j) for j in i)+'\n' for i in centroid_points)
    

# def stop_criterion2(old_centroids,new_centroids,t):
#     """
#     If a centroid moves by more than 0.001 in a given iteration, return False (and therefore continue to iterate)
#     """
#     k=len(old_centroids)
#     deltas=[0]*k
#     flag=0
#     for i in range(k):
#         deltas[i]=find_distance(old_centroids[i],new_centroids[i])
#     if max(deltas)>t:
#         flag=1
#     return flag

def stop_criterion(centroid_points_old, centroid_points_new,T):
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    Diff = [abs(x-y) for x, y in zip(oldvalue, newvalue)]
    Flag = True
    for i in Diff:
        if(i>T):
            Flag = False
            break
    return Flag
        

#Update centroids iteratively
i=0 #Track which iteration we're on
while(1):
    centroid_points_old=centroid_points[:]
    
    print 'iteration '+str(i)+':'
    with mr_job.make_runner() as runner:
        runner.run()
        #stream output
        for line in runner.stream_output():
            key,value=mr_job.parse_output_line(line)
            print key,value[0],value[1][0]
            centroid_points[key]=value[1]
    print '\n'
    i+=1
    
    if stop_criterion(centroid_points_old,centroid_points,0.001):
    #if i>4:
        break
    sleep(1) #so warnings don't print funny
    
print 'DONE'
# NEED TO ADD EVALUATION CRITERIA HERE



iteration 0:
CENTROIDS LENGTH
4
0 [436.0, 55.0, 8.0, 34.0] 0.0265668676155
1 [316.0, 4.0, 11.0, 69.0] 0.0159074634313
2 [0.0, 32.0, 30.0, 0.0] 0.150313905584
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 1:


CENTROIDS LENGTH
4
0 [148.0, 7.0, 4.0, 24.0] 0.0269465437596
1 [603.0, 1.0, 8.0, 75.0] 0.0121415185199
2 [1.0, 83.0, 37.0, 4.0] 0.132561177813
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 2:


CENTROIDS LENGTH
4
0 [99.0, 1.0, 4.0, 42.0] 0.0321593776955
1 [653.0, 2.0, 8.0, 57.0] 0.0114547561475
2 [0.0, 88.0, 37.0, 4.0] 0.131006933241
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 3:


CENTROIDS LENGTH
4
0 [53.0, 1.0, 4.0, 59.0] 0.0368272178281
1 [699.0, 2.0, 8.0, 40.0] 0.011527248443
2 [0.0, 88.0, 37.0, 4.0] 0.131006933241
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 4:


CENTROIDS LENGTH
4
0 [29.0, 0.0, 4.0, 76.0] 0.0352120258925
1 [723.0, 3.0, 8.0, 23.0] 0.0120271898909
2 [0.0, 88.0, 37.0, 4.0] 0.131006933241
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 5:


CENTROIDS LENGTH
4
0 [23.0, 0.0, 2.0, 71.0] 0.0356037204133
1 [729.0, 3.0, 10.0, 28.0] 0.0123697875455
2 [0.0, 88.0, 37.0, 4.0] 0.131006933241
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 6:


CENTROIDS LENGTH
4
0 [21.0, 0.0, 2.0, 67.0] 0.0345415688004
1 [731.0, 3.0, 10.0, 32.0] 0.0126726190434
2 [0.0, 88.0, 37.0, 4.0] 0.131006933241
3 [0.0, 0.0, 5.0, 0.0] 0.0


iteration 7:


CENTROIDS LENGTH
4
0 [20.0, 0.0, 2.0, 66.0] 0.034037269403
1 [732.0, 3.0, 10.0, 33.0] 0.0127858790003
2 [0.0, 88.0, 37.0, 4.0] 0.131006933241
3 [0.0, 0.0, 5.0, 0.0] 0.0


DONE


###End of Submission

#TODO:
- Clean up output of iterative jobs
- Figure out how to run with different initializations (including changing value of K)
- Add purity calculations
- Finalize commentary, and finish part 1
